<a href="https://colab.research.google.com/github/LucasFelipeNunes/Analista-de-Conversas-de-WhatsApp/blob/main/Analista_de_Conversas_de_WhatsApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📄 Analista de Conversas do WhatsApp

Olá! Este programa tem como objetivo analisar estatísticas detalhadas de uma conversa exportada (sem mídias) do WhatsApp, tendo sido desenvolvido em Python na plataforma do Google Colab. A análise inclui aspectos como volume de mensagens, distribuição temporal, uso de palavras, emojis, mídias e outros indicadores de comportamento dos participantes.

Para utilizar este programa, você pode abrir este código com o botão no canto superior esquerdo e salvá-lo no seu Drive, e, na mesma pasta em que você salvar o programa, salvar o arquivo em txt da conversa exportada do WhatsApp, com o nome `conversa.txt`.

## 📥 Entrada

O programa espera um arquivo de texto no formato padrão de exportação do WhatsApp (sem mídia), em que cada linha segue o formato:

```
dd/mm/aaaa hh:mm - Nome: Mensagem
```

Exemplo:

```
20/04/2024 14:53 - João: Olá, tudo bem?
```

## 📤 Saída

Os resultados são exibidos diretamente no console, organizados em seções com listagens ordenadas de acordo com a frequência ou volume.

## Importação das Bibliotecas

In [ ]:
import re
import numpy as np
from collections import defaultdict, Counter
from emoji import demojize, is_emoji
from google.colab import drive

## Conexão com o Drive

In [ ]:
drive.mount('/content/drive')
arquivo = "/content/drive/MyDrive/conversa.txt"

## Funções Auxiliares

In [ ]:
def detectar_periodo(hora):
    h = int(hora.split(':')[0])
    if h < 6:
        return 'madrugada'
    elif h < 12:
        return 'manhã'
    elif h < 18:
        return 'tarde'
    return 'noite'

def limpar_mensagem(msg):
    return msg.lower().strip()

def filtrar_palavras(palavras):
    return [
        p for p in palavras
        if len(p) >= 5
        and not re.fullmatch(r'(ha|he|hi|ho|hu|rs)+', p)
        and not re.fullmatch(r'[k]{3,}', p)
        and not re.fullmatch(r'\d+', p)
        and not p.startswith("http")
    ]


def extrair_emojis(texto):
    return [char for char in texto if is_emoji(char)]

## Declaração das Variáveis

In [ ]:
padrao_msg = re.compile(r'^(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}) - ([^:]+): (.+)$')

total_mensagens = total_palavras = 0
mensagens_por_usuario = defaultdict(int)
palavras_por_usuario = defaultdict(int)
palavras_comuns = Counter()
palavras_comuns_por_usuario = defaultdict(Counter)
ultimas_mensagens_por_dia = {}
maior_mensagem = ''
pessoa_maior_mensagem = ''
mensagens_por_periodo = defaultdict(int)
mensagens_por_usuario_periodo = defaultdict(lambda: defaultdict(int))
midias_por_usuario = defaultdict(int)
apagadas_por_usuario = defaultdict(int)
emojis_geral = Counter()
emojis_por_usuario = defaultdict(Counter)
caracteres_por_mensagem = defaultdict(int)

## Algoritmo Principal

In [ ]:
with open(arquivo, encoding='utf-8') as f:
    for linha in f:
        linha = linha.strip()
        match = padrao_msg.match(linha)
        if not match:
            continue

        data, hora, nome, msg = match.groups()
        msg_lower = limpar_mensagem(msg)

        mensagens_por_usuario[nome] += 1
        ultimas_mensagens_por_dia[data] = nome
        total_mensagens += 1

        periodo = detectar_periodo(hora)
        mensagens_por_periodo[periodo] += 1
        mensagens_por_usuario_periodo[nome][periodo] += 1

        if "<mídia oculta>" in msg_lower:
            midias_por_usuario[nome] += 1
            continue

        if "mensagem apagada" in msg_lower:
            apagadas_por_usuario[nome] += 1
            continue

        if len(msg) > len(maior_mensagem):
            maior_mensagem = msg
            pessoa_maior_mensagem = nome

        emojis = extrair_emojis(msg)
        if emojis:
            emojis_geral.update(emojis)
            emojis_por_usuario[nome].update(emojis)

        caracteres_por_mensagem[nome] += len(msg)

        palavras = filtrar_palavras(re.findall(r'\b\w+\b', msg_lower))
        for p in palavras:
            palavras_comuns[p] += 1
            palavras_comuns_por_usuario[nome][p] += 1
            palavras_por_usuario[nome] += 1
            total_palavras += 1

## Impressão dos Relatórios

In [ ]:
def imprimir_relatorios():
    aux = 0
    print(f"Total de mensagens: {total_mensagens}\nMensagens por usuário:")
    for usuario, total in sorted(mensagens_por_usuario.items(), key=lambda x: x[1], reverse=True):
        aux += 1
        media = caracteres_por_mensagem[usuario] / total
        print(f"{aux}. {usuario}: {total}, média de caracteres {media:.2f}")

    aux = 0
    print(f"\nTotal de palavras: {total_palavras}\nPalavras por usuário:")
    for usuario, total in sorted(palavras_por_usuario.items(), key=lambda x: x[1], reverse=True):
        aux += 1
        print(f"{aux}. {usuario}: {total}")

    aux = 0
    print("\nTop 5 palavras:")
    for palavra, count in palavras_comuns.most_common(5):
        aux += 1
        print(f"{aux}. {palavra}: {count}")

    aux = 0
    print("\nTop 5 palavras por usuário:")
    for usuario, contador in palavras_comuns_por_usuario.items():
        lista = [f"{palavra} ({quantidade})" for palavra, quantidade in contador.most_common(5)]
        print(f"\n{usuario}:")
        for palavra in lista:
            aux += 1
            print(f"{aux}. {palavra}")
        aux = 0

    print("\nMensagens por período:")
    for periodo, total in mensagens_por_periodo.items():
        print(f"{periodo.capitalize()}: {total}")

    aux = 0
    print("\nUsuários por período:")
    for periodo in ['madrugada', 'manhã', 'tarde', 'noite']:
        print(f"\n{periodo.capitalize()}:")
        usuarios = mensagens_por_usuario_periodo.items()
        lista = sorted(((u, p[periodo]) for u, p in usuarios if periodo in p), key=lambda x: x[1], reverse=True)
        for usuario, total in lista:
            aux += 1
            print(f"{aux}. {usuario}: {total}")
        aux = 0

    print(f"\nMaior mensagem: {len(maior_mensagem)} caracteres, de {pessoa_maior_mensagem}")

    aux = 0
    print("\nMídias por usuário:")
    for usuario, total in sorted(midias_por_usuario.items(), key=lambda x: x[1], reverse=True):
        aux += 1
        print(f"{aux}. {usuario}: {total}")

    aux = 0
    print("\nMensagens apagadas por usuário:")
    for usuario, total in sorted(apagadas_por_usuario.items(), key=lambda x: x[1], reverse=True):
        aux += 1
        print(f"{aux}. {usuario}: {total}")

    aux = 0
    print("\nVácuos por dia:")
    vacuos = Counter(ultimas_mensagens_por_dia.values())
    for autor, dias in vacuos.most_common():
        aux += 1
        print(f"{aux}. {autor} ficou no vácuo em {dias} dias.")

    aux = 0
    print("\nTop emojis (geral):")
    for emoji, count in emojis_geral.most_common(5):
        aux += 1
        print(f"{aux}. {emoji} - {count}x")

    aux = 0
    print("\nTop emojis por usuário:")
    for usuario, contador in emojis_por_usuario.items():
        print(f"\n{usuario}:")
        for emoji, count in contador.most_common(5):
            aux += 1
            print(f"{aux}. {emoji} - {count}x")
        aux = 0

    aux = 0
    print("\nCaracteres por usuário:")
    for usuario, total in sorted(caracteres_por_mensagem.items(), key=lambda x: x[1], reverse=True):
        aux += 1
        print(f"{aux}. {usuario}: {total}")

imprimir_relatorios()